In [ ]:
!pip install rasterio pyvista trimesh shapely

import rasterio
import numpy as np
import pyvista as pv
import trimesh

# Ruta al raster
ruta = '/content/gebco_2025.tif'

# --- Leer el GeoTIFF ---
with rasterio.open(ruta) as src:
    elev = src.read(1).astype(float)  # matriz de elevación
    transform = src.transform

# Crear coordenadas X, Y
rows, cols = elev.shape
x = np.arange(cols) * transform.a + transform.c
y = np.arange(rows) * transform.e + transform.f
X, Y = np.meshgrid(x, y)

# --- Crear grilla estructurada ---
grid = pv.StructuredGrid(X, Y, elev)

# --- Convertir StructuredGrid → PolyData (triangular) ---
surf = grid.extract_surface().triangulate()

# Extraer vértices y caras
vertices = surf.points
faces = surf.faces.reshape(-1, 4)[:, 1:]  # cada fila: (3, v0, v1, v2)

# Crear malla con Trimesh
mesh = trimesh.Trimesh(vertices=vertices, faces=faces, process=False)

# --- Exportar a glTF (.glb) ---
mesh.export("gebco_2025.glb")
print("✅ Exportado a gebco_2025.glb")


✅ Exportado a gebco_2025.glb


In [9]:
!apt-get update
!apt-get install -y gdal-bin

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://cli.github.com/packages stable InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,209 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,942 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,297 kB]
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubunt

In [10]:
# Convertir a 8-bit escalando valores
!gdal_translate -of VRT -ot Byte -scale /content/gebco_2025.tif temp.vrt

# Luego generar los tiles
!gdal2tiles.py -z 0-8 temp.vrt tiles/

Input file size is 3739, 2083
Warning 1: for band 1, nodata value has been clamped to 0, the original value being out of range.
Generating Base Tiles:
0...10...20...30...40...50...60...70...80...90...100 - done.
Generating Overview Tiles:
0...10...20...30...40...50...60...70...80...90...100 - done.


In [11]:
# Comprimir la carpeta tiles en un zip
!zip -r tiles.zip tiles

# Descargar el zip
from google.colab import files
files.download("tiles.zip")

  adding: tiles/ (stored 0%)
  adding: tiles/googlemaps.html (deflated 74%)
  adding: tiles/0/ (stored 0%)
  adding: tiles/0/0/ (stored 0%)
  adding: tiles/0/0/0.png (deflated 24%)
  adding: tiles/8/ (stored 0%)
  adding: tiles/8/82/ (stored 0%)
  adding: tiles/8/82/93.png (deflated 12%)
  adding: tiles/8/82/97.png (deflated 0%)
  adding: tiles/8/82/95.png (stored 0%)
  adding: tiles/8/82/94.png (stored 0%)
  adding: tiles/8/82/90.png (deflated 32%)
  adding: tiles/8/82/96.png (stored 0%)
  adding: tiles/8/82/91.png (deflated 76%)
  adding: tiles/8/82/92.png (stored 0%)
  adding: tiles/8/82/89.png (deflated 5%)
  adding: tiles/8/83/ (stored 0%)
  adding: tiles/8/83/93.png (deflated 11%)
  adding: tiles/8/83/97.png (stored 0%)
  adding: tiles/8/83/95.png (deflated 6%)
  adding: tiles/8/83/94.png (deflated 30%)
  adding: tiles/8/83/90.png (deflated 14%)
  adding: tiles/8/83/96.png (stored 0%)
  adding: tiles/8/83/91.png (deflated 81%)
  adding: tiles/8/83/92.png (deflated 61%)
  adding: 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>